In [128]:
import os
import requests
import pandas as pd
from openai import OpenAI
from datetime import datetime
from twilio.rest import Client

from dotenv import load_dotenv

load_dotenv()

True

In [82]:
def refresh_access_token():
    res = requests.post(
        "https://www.strava.com/oauth/token",
        data={
            "client_id": os.getenv("STRAVA_CLIENT_ID"),
            "client_secret": os.getenv("STRAVA_CLIENT_SECRET"),
            "refresh_token": os.getenv("STRAVA_REFRESH_TOKEN"),
            "grant_type": "refresh_token",
        }
    )
    print(res.status_code)
    print(res.json())
    res.raise_for_status()
    return res.json()["access_token"]

In [83]:
def get_strava_activities(access_token, per_page=5):
    url = "https://www.strava.com/api/v3/athlete/activities"
    headers = {"Authorization": f"Bearer {access_token}"}
    params = {"per_page": per_page}
    
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()
    return response.json()

In [ ]:
# get access token
token_data = refresh_access_token()
# call up strava
strava_data = get_strava_activities(access_token=token_data, per_page=10)


200
{'token_type': 'Bearer', 'access_token': '51e877d88b10b5f79b5a1f4c763dd44e4a3bdd24', 'expires_at': 1748647824, 'expires_in': 21600, 'refresh_token': 'c280cb6536c00d9cbcd05b660423cb5ee91e34c8'}


In [86]:
strava_results = pd.DataFrame(strava_data)

In [109]:
strava_results['moving_time_mins'] = round(strava_results['moving_time']/60, 1)
strava_results['distance_miles'] = round(strava_results['distance']/1609.34, 2)
strava_results['elevation_gain_ft'] = round(strava_results['total_elevation_gain']*3.281, 1)


In [110]:
filtered_strava_results = strava_results[['distance_miles', 'moving_time_mins', 'elevation_gain_ft', 'type', 'start_date_local', 'average_speed', 'average_heartrate', 'max_heartrate']]

In [111]:
filtered_strava_results

,distance_miles,moving_time_mins,elevation_gain_ft,type,start_date_local,average_speed,average_heartrate,max_heartrate
0,3.02,40.5,112.9,Run,2025-05-28T17:35:23Z,1.995,145.0,158.0
1,8.53,97.3,369.4,Run,2025-05-26T11:52:59Z,2.352,169.3,187.0
2,4.99,55.8,0.0,Run,2025-05-23T09:14:03Z,2.394,161.9,180.0
3,6.01,68.7,168.0,Run,2025-05-20T07:30:00Z,2.347,163.2,180.0
4,4.12,92.2,874.1,Walk,2025-05-19T19:45:26Z,1.200,102.8,161.0
5,5.50,73.8,232.3,Run,2025-05-18T17:34:04Z,1.999,149.7,170.0
6,4.51,52.8,34.8,Run,2025-05-16T10:10:11Z,2.293,174.4,192.0
7,2.55,29.1,207.0,Run,2025-05-13T12:47:05Z,2.354,163.2,182.0
8,6.04,69.5,207.0,Run,2025-05-11T12:53:14Z,2.334,169.3,189.0
9,3.54,42.0,143.4,Run,2025-05-06T09:55:27Z,2.263,172.2,188.0


In [126]:
# Sort and get the latest 5 activities (already sorted in your screenshot, but just in case)
df = filtered_strava_results.sort_values("start_date_local", ascending=False).head(5)

# Format each activity for the prompt
activity_lines = []
for _, row in df.iterrows():
    date = pd.to_datetime(row['start_date_local']).strftime('%Y-%m-%d')
    activity_type = row['type']
    distance = f"{row['distance_miles']:.2f} mi"
    time = f"{row['moving_time_mins']:.1f} min"
    elev = f"{row['elevation_gain_ft']:.0f} ft"
    avg_hr = f"{row['average_heartrate']:.0f}" if not pd.isnull(row['average_heartrate']) else "N/A"
    max_hr = f"{row['max_heartrate']:.0f}" if not pd.isnull(row['max_heartrate']) else "N/A"
    line = f"{date}: {activity_type} - {distance}, {time}, {elev} gain, Avg HR: {avg_hr}, Max HR: {max_hr}"
    activity_lines.append(line)

# add date
today = datetime.now().strftime("%Y-%m-%d")

# Create the prompt
activity_str = "\n".join(activity_lines)
prompt = (
    f"Today is {today}.\n"
    "Here are my last 5 Strava activities:\n"
    f"{activity_str}\n\n"
    """Please send me a good morning text message that includes the following:
    - an encouraging message about my recent strava activity
    - today's date
    - an interesting fact about san francisco native plants, animals, or history"""
)

print(prompt)

Today is 2025-05-30.
Here are my last 5 Strava activities:
2025-05-28: Run - 3.02 mi, 40.5 min, 113 ft gain, Avg HR: 145, Max HR: 158
2025-05-26: Run - 8.53 mi, 97.3 min, 369 ft gain, Avg HR: 169, Max HR: 187
2025-05-23: Run - 4.99 mi, 55.8 min, 0 ft gain, Avg HR: 162, Max HR: 180
2025-05-20: Run - 6.01 mi, 68.7 min, 168 ft gain, Avg HR: 163, Max HR: 180
2025-05-19: Walk - 4.12 mi, 92.2 min, 874 ft gain, Avg HR: 103, Max HR: 161

Please send me a good morning text message that includes the following:
    - an encouraging message about my recent strava activity
    - today's date
    - an interesting fact about san francisco native plants, animals, or history


In [127]:
client = OpenAI()

# NOTES ON PROMPT ENGINEERING
# -- If you say it's an expert on history of SF as well as local flora/fauna and then ask for a fact, every fact will be about GGP being bigger than central park
# -- chatGPT seems to hallucinate a lot more via API than in the GUI even for the exact same model/prompt.

response = client.responses.create(
    model="gpt-4.1",
    instructions="""You are a supportive life coach and personal trainer named Zeus Groos. You are also knowledgeable
        about the history of San Francisco as well as local flora and fauna. You do not make up any information and only say 
        things that you know to be true. Your trainee, Zach, is training for the New York Marathon on November 3 2025.
        He is doing a half marathon training program that ends on June 29 and then he'll begin the full marathon training program.""",
    input=prompt,
)

print(response.output_text)

Good morning, Zach! 🌞  
Today is May 30, 2025.

I’m proud of your consistency and perseverance—your last five Strava activities show a steady commitment to your half marathon training. That recent 8.5-mile run at a strong pace and heart rate is a great sign of your progress. Keep listening to your body and trust in your training; you’re building serious strength for your marathon goals!

Here’s a little local inspiration: Did you know that the San Francisco garter snake is one of the most beautiful and endangered snakes in North America? Found only in the San Francisco Peninsula, its striking red, black, and turquoise stripes stand out just like your dedication stands out in your training log!  

Let’s keep moving forward—one step at a time! 🏃‍♂️✨


In [140]:
# Load environment variables from .env file
load_dotenv()

account_sid = os.getenv("TWILIO_ACCOUNT_SID")
auth_token = os.getenv("TWILIO_AUTH_TOKEN")
twilio_number = os.getenv("TWILIO_PHONE_NUMBER")
my_cell = os.getenv("MY_CELL_NUMBER")
# my_cell = "+17147560044"

twilio_client = Client(account_sid, auth_token)

message = twilio_client.messages.create(
    body="Sent from your Twilio trial account - Hello from Twilio!",
    from_=twilio_number,
    to=my_cell
)

print(f"Message sent! SID: {message.sid}")
print(f"Message status: {message.status}")

Message sent! SID: SM1ca699bcd8ceb8f14052d08e60914193
Message status: queued


In [141]:
# Fetch the message again to check delivery status
fetched_message = twilio_client.messages(message.sid).fetch()
print(f"Final message status: {fetched_message.status}")
print(f"Error code: {fetched_message.error_code}")
print(f"Error message: {fetched_message.error_message}")

Final message status: undelivered
Error code: 30032
Error message: None
